In [ ]:
!pip install paddlepaddle
!pip install paddleocr

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 47829, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 47829 (delta 64), reused 111 (delta 32), pack-reused 47649
Receiving objects: 100% (47829/47829), 345.42 MiB | 23.76 MiB/s, done.
Resolving deltas: 100% (33595/33595), done.
Updating files: 100% (1892/1892), done.


In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import cv2
import glob

In [ ]:
!unzip test_images.zip

Archive:  test_images.zip
   creating: test_images/
  inflating: test_images/map_1.png   
  inflating: test_images/map_2.png   
  inflating: test_images/map_3.png   
  inflating: test_images/map_4.png   
  inflating: test_images/map_5.png   
  inflating: test_images/map_7.png   
  inflating: test_images/map_8.png   
  inflating: test_images/map_10.png  
  inflating: test_images/Cb1.png     
  inflating: test_images/Cb2.png     
  inflating: test_images/Cb3.png     
  inflating: test_images/Cb4.png     
  inflating: test_images/Cb5.png     
  inflating: test_images/Cb6.png     
  inflating: test_images/Cb7.png     
  inflating: test_images/Cb8.png     
  inflating: test_images/Cb9.png     
  inflating: test_images/Cb10.png    


In [ ]:
ocr_model = PaddleOCR(lang='en')

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:13<00:00, 293kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:14<00:00, 725kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:04<00:00, 518kiB/s]

[2024/03/16 17:03:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

In [ ]:
img_path = '/content/test_images/'

In [ ]:
img_paths = glob.glob(img_path + '*.png')

In [ ]:
results = {}

In [ ]:
map_data = {
    "map_265": {
        "value1":{
            "color": (1,2,3),  # BGR format
            "lower_bound": 10,
            "upper_bound": 40,
        }
    },
}
print(map_data)

# def add_map(map_id, value_id, color, lower_bound, upper_bound):
#     map_data[map_id][value_id] = {
#         "color": color,
#         "lower_bound": lower_bound,
#         "upper_bound": upper_bound
#     }

# def add_new_map(map_id, value_id, color, lower_bound, upper_bound):
#     map_data[map_id][value_id] = {
#         "color": color,
#         "lower_bound": lower_bound,
#         "upper_bound": upper_bound
#     }

def add_new_element(map_data, map_num, value_id, color, lower_bound, upper_bound):
  if map_num not in map_data:
    map_data[map_num] = {}
  if value_id not in map_data[map_num]:
    map_data[map_num][value_id] = {
        "color": color,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "value": (lower_bound+upper_bound)//2,
    }

{'map_265': {'value1': {'color': (1, 2, 3), 'lower_bound': 10, 'upper_bound': 40}}}


In [ ]:
# Function to check if a string is a digit
def is_digit(s):
    return s.isdigit()

# Function to convert bound variables to doubles and extract units
def convert_to_doubles(lower_bound, upper_bound):
    # Check if last character of lower_bound is not a digit
    if not is_digit(lower_bound[-1]):
        units = lower_bound[-1]
        lower_bound = float(lower_bound[:-1])
    else:
        units = 'u'
        lower_bound = float(lower_bound)

    # Check if last character of upper_bound is not a digit
    if not is_digit(upper_bound[-1]):
        units = upper_bound[-1]
        upper_bound = float(upper_bound[:-1])
    else:
        units = 'u'
        upper_bound = float(upper_bound)

    return lower_bound, upper_bound, units

In [ ]:
from google.colab.patches import cv2_imshow
import logging

# Suppress PaddleOCR warning and debug messages
logging.getLogger("PaddleOCR").setLevel(logging.ERROR)

for img_path in img_paths:
    result = ocr_model.ocr(img_path)
    results[img_path] = result

def num_there(s):
    return any(i.isdigit() for i in s)

def contains_hyphen(text):
    """
    Check if the given text contains the character '-'.

    Args:
        text (str): The text to check.

    Returns:
        bool: True if the text contains '-', False otherwise.
    """
    return '-' in text

# numerical_text_info = []
complete_data = []
# j=0

for img_path, result in results.items():
  numerical_text_info = []
  print(f"OCR results for {img_path}:")
  image = cv2.imread(img_path)
  for line in result:
    # print(line)
    # print()
    for word in line:
      # print(word[0][0])
      # print(word[0][0][0])
      # print(word[1][0])
      text = word[1][0]
      x, y, w, h = word[0][0][0], word[0][0][1], abs(word[0][0][0]-word[0][1][0]), abs(word[0][1][1]-word[0][2][1])
      # print(x, y, w, h)
      #  and contains_hyphen(text)==1
      if num_there(text)==1:
        # print(type(text))
        # print(f"Text: {text}, Bounding Box: {x}, {y}, {w}, {h}")
        # cv2.rectangle(images, (x, y), (x + w, y + h), (0, 255, 0), 2)

        x1 = int(x)
        y1 = int(y+h//2)
        x2 = int(x)
        y2 = int(y +h//2- 1)
        # x1 = x
        # x2 = x
        # y1 = y + h // 2
        # y2 = y + h // 2 - 1

        while all(image[y1, x1]==image[y2, x2]):
            x2-=1

        # x2-=5
        x3=x2
        # cv2.rectangle(image, (x2, y2), (x2+5, y2+5), (0, 255, 0), 2)

        while all(image[y1, x1]==image[y2, x3]):
           x3-=1

        x2 = (x2+x3)//2

        color = image[y2,x2]

        blue, green, red = color
        # print(f"Blue: {blue}, Green: {green}, Red: {red}")

        # values = text.split("-")
        numerical_text_info += [text] + [red, green, blue]
        # print(numerical_text_info)
        # numerical_text_info += [text] + [blue, green, red]
        # print(numerical_text_info)

        # color = image[y,x-25] # have to edit this to get the exact colour.
        # print(f'Color at ({x}, {y}): {color}')

  # cv2_imshow(image)

  i=0
  value_id = 0
  while i<len(numerical_text_info):
    value_id += 1

    values = numerical_text_info[i].split("-")
    # print(values)

    # print(type(numerical_text_info[i]))
    if len(values)>1:
      lower_bound = values[0]
      upper_bound = values[1]
    else:
      lower_bound = values[0]
      upper_bound = values[0]

    converted_lower_bound, converted_upper_bound, units = convert_to_doubles(lower_bound, upper_bound)
    info = [value_id, numerical_text_info[i+1], numerical_text_info[i+2], numerical_text_info[i+3], (converted_lower_bound+converted_upper_bound)/2, units]
    # print(info)

    # add_map(map_id, value_id, (blue, green, red), lower_bound, upper_bound)
    # print(map_data)

    # map_name = "map_287"
    map_name = img_path.split("/")[-1].split(".")[0]
    info = [map_name] + info
    complete_data += [info]

    # formatted_data = [data[0][0]]  # First element is the map name
    # for item in data:
    #   formatted_data.append(item[1:])
    # print(info)
    # add_new_element(map_data, map_name, value_id, (blue, green, red), lower_bound, upper_bound)
    # if map_name in map_data:
    #     # map_data[map_name][value_id] = {
    #     #     "color": (blue, green, red),
    #     #     "lower_bound": lower_bound,
    #     #     "upper_bound": upper_bound,
    #     # }
    # else:
    #     map_data[map_name] = {
    #         value_id: {
    #             "color": (blue, green, red),
    #             "lower_bound": lower_bound,
    #             "upper_bound": upper_bound,
    #         }
    #     }

    i+=4

# print(map_data)
print(complete_data)
  # print(color)
  # cv2.rectangle(images, (x2, y2), (x2+5, y2+5), (0, 255, 0), 2)

# os.remove(filename)
# # cv2_imshow(cv2.cvtColor(images, cv2.COLOR_BGR2RGB))
# cv2_imshow(images)

[2024/03/16 17:03:08] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/03/16 17:03:09] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.27518367767333984
[2024/03/16 17:03:09] ppocr DEBUG: rec_res num  : 8, elapsed : 0.6554462909698486
[2024/03/16 17:03:09] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/03/16 17:03:09] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.12295126914978027
[2024/03/16 17:03:10] ppocr DEBUG: rec_res num  : 8, elapsed : 0.5491740703582764
[2024/03/16 17:03:10] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/03/16 17:03:10] ppocr DEBUG: dt_boxes num : 6, elapsed : 0.1292562484741211
[2024/03/16 17:03:11] ppocr DEBUG: rec_res num  : 6, elapsed : 0.8658435344696045
[2024/03/16 17:03:11] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during th

In [ ]:
with open('/content/output.txt', 'w') as f:
    # Loop through the list and write each element to the file
    for item in complete_data:
        f.write(str(item) + '\n')

In [ ]:
results

{'/content/test_images/Cb4.png': [[[[[58.0, 25.0],
     [318.0, 25.0],
     [318.0, 39.0],
     [58.0, 39.0]],
    ('The Number of churches in the USA', 0.9746752381324768)],
   [[[618.0, 58.0], [656.0, 61.0], [655.0, 75.0], [617.0, 73.0]],
    ('Value', 0.9978026151657104)],
   [[[652.0, 107.0], [684.0, 110.0], [683.0, 124.0], [651.0, 122.0]],
    ('400k', 0.9795717000961304)],
   [[[652.0, 174.0], [684.0, 174.0], [684.0, 188.0], [652.0, 188.0]],
    ('395k', 0.9968050718307495)],
   [[[652.0, 235.0], [684.0, 238.0], [683.0, 252.0], [651.0, 250.0]],
    ('390k', 0.971998929977417)],
   [[[652.0, 303.0], [685.0, 303.0], [685.0, 316.0], [652.0, 316.0]],
    ('385k', 0.9935635328292847)],
   [[[652.0, 365.0], [684.0, 368.0], [683.0, 382.0], [651.0, 380.0]],
    ('380k', 0.978151798248291)],
   [[[651.0, 431.0], [684.0, 431.0], [684.0, 444.0], [651.0, 444.0]],
    ('375k', 0.9701771140098572)]]],
 '/content/test_images/Cb3.png': [[[[[43.0, 24.0],
     [305.0, 24.0],
     [305.0, 41.0],
  